### Mandy's Fruit Identification Project

In [ ]:
import pandas as pd
import livelossplot
import keras
from keras import Sequential

import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential

from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

from keras.layers import Dense
from keras.layers import Dropout

from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay 



In [ ]:
pip install livelossplot

In [ ]:
# Fitting images
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,)

test_datagen = ImageDataGenerator(rescale = 1./255,)

Note - the assignment states to divide your dataset here into training and testing, however in this dataset each training and testing already have their own folders for pulling in with a 75% training and 25% testing.

## About the dataset
The dataset Fruits 360 has a Total number of images: 90483. It has 131 classes (also note there is a multiple fruit set that I haven't used in my modelling at this time)

In [ ]:
# Locate and assign Training Set - noted that I must switch the slash direction for syntax within paths
training_set = train_datagen.flow_from_directory('C:/Users/hyppi/OneDrive/Bootcamp/assessments Python/Module8 - CNN and RNN/fruits-360_dataset/fruits-360/Training',
                                                 target_size = (100, 100),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 shuffle = True)

In [ ]:
test_set = test_datagen.flow_from_directory('C:/Users/hyppi/OneDrive/Bootcamp/assessments Python/Module8 - CNN and RNN/fruits-360_dataset/fruits-360/Test',
                                            target_size = (100, 100),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            shuffle = False)

In [ ]:
x_test_list = []
y_test_list = []
batch_index = 0

while batch_index <= test_set.batch_index:
    x_test,  y_test = test_set.next()
    x_test_list.extend(x_test)
    y_test_list.extend(y_test)
    batch_index = batch_index + 1# now, data_array is the numeric data of whole images
  
print(x_test_list)
print(y_test_list)
print(len(x_test_list))
print(len(y_test_list))

In [ ]:
y_test

In [ ]:
#Next to train our model

# Initialising the CNN
fruit_model = Sequential()

# Step 1 - Convolution
fruit_model.add(Conv2D(16, (2, 2), input_shape = (100, 100, 3), activation = 'relu'))

# Step 2 - Pooling
fruit_model.add(MaxPooling2D(pool_size = (2, 2)))

# Another convolutional layer
fruit_model.add(Conv2D(32, (2, 2), input_shape = (100, 100, 3), activation = 'relu'))

# Another pooling layer with 2x2
fruit_model.add(MaxPooling2D(pool_size = (2, 2)))

# Two more 2D Convolutional layers with 64 unitix and size 2x2 and relu
fruit_model.add(Conv2D(64, (2, 2), input_shape = (100, 100, 3), activation = 'relu'))
fruit_model.add(Conv2D(64, (2, 2), input_shape = (100, 100, 3), activation = 'relu'))

# Another pooling layer with 2x2
fruit_model.add(MaxPooling2D(pool_size = (2, 2)))

# Add a dropout layer with 0.3  rate
fruit_model.add(Dropout(0.3))

# Step 3 - Flattening
fruit_model.add(Flatten())

# Step 4 - Full connection ANN layers
# Add fully connected layer with 150 units with ReLU and then a dropout layer with a 0.4 rate
fruit_model.add(Dense(units=150, activation='relu'))

#another dropout layer with 0.5 rate
fruit_model.add(Dropout(rate=0.4))

# NOTE - I thought that the output layer was supposed to be the number of classes (131 in this dataset)  however the assignment states to use 81 units - I will inquire on this at next class

# Final Output layer fully connected with 81 units and softmax activation.
fruit_model.add(Dense(units=131, activation='softmax'))

In [ ]:
# Compiling the CNN
fruit_model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
class_names = test_set.class_indices.keys()
print(class_names)

In [ ]:
fruit_model.summary()



## Training to classify fruit

### Here I recieved the error ValueError: The `batch_size` argument must not be specified when using a generator or Sequence as an input.

###  This led into a different value error saying that the expected size for my dense_2 output layer of 131, was instead showing as a shape of 1, 

### This led down a bit of a rabbit hole trying to find the solution as essentially it seemed to be binary but our data was catagorical, at first the thought was we might need to one-hot or to_catagorical

### In the end however of looking one more time I realized that the example notebook pulled in class_mode of binary and I had missed changing that to categorical, once change it then  worked properly.


In [ ]:
from livelossplot import PlotLossesKeras

fruit_model.fit(training_set,
               steps_per_epoch = 32,
               epochs = 100,
               validation_data=(test_set),
               callbacks=[PlotLossesKeras()],
               validation_steps = 20,
               verbose=1)

In [ ]:
#pull the loss and accuracy from the model
loss, accuracy = fruit_model.evaluate(test_set)

In [ ]:
loss

In [ ]:
accuracy

In [ ]:

#Make predictions on the test data with the trained model
y_prediction = np.argmax(fruit_model.predict(test_set), axis=1)
y_test1 = np.argmax(y_test_list, axis=1)

In [ ]:
confusion_matrix = confusion_matrix(list(y_test1), list(y_prediction))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix)

fig, ax = plt.subplots(figsize=(40,40))
disp.plot(ax=ax)
plt.show()

## The result is showing for me  at 86% accuracy which feels pretty good with 131 classes of fruits and vegetables that might have similar appearance.